In [31]:
from pathlib import Path
from pandarallel import pandarallel

pandarallel.initialize(progress_bar=True)

DATASETS = list(Path("../data").rglob("predictions-experiment*.json"))

INFO: Pandarallel will run on 6 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [32]:
for dataset in DATASETS:
    print(dataset)

../data/predictions-experiment-2.json
../data/predictions-experiment-1.json


### Functions for filling masks

We need to define functions to replace the masks with correct words

In [33]:
import pandas as pd

def _correct_row(row: pd.Series) -> str | None:
    corrected = row["masked"]

    try:
        for fix in row["replace"]:
            corrected = corrected.replace("[MASK]", fix, 1)
    except TypeError:
        return None
    
    return corrected

def correct_row(df: pd.DataFrame) -> pd.DataFrame:
    df["corrected"] = None

    index = df[df["masked"].notnull()].index
    _df = df.loc[index]

    _df["corrected"] = _df.apply(_correct_row, axis=1)
    df.loc[index] = _df
    
    return df

### Calculate avg. edit distance

- distance against `errored` sentence
- distance against `corrected` sentence

In [34]:
from nltk import edit_distance

def calc_edit_distance(row: pd.Series) -> tuple[int, int]:
    base = edit_distance(row["sentence"], row["error"])

    if row["corrected"] is None:
        return base, base

    return base, edit_distance(row["sentence"], row["corrected"])

In [35]:
import random

for dataset in DATASETS:

    df = pd.read_json(dataset, orient="records", lines=True)
    
    df = correct_row(df)
    
    # calculate edit distance 
    print("Calculating edit distance...")
    df[['ED', 'ED_corrected']] = df.parallel_apply(calc_edit_distance, axis=1, result_type='expand')
    
    print("Edit distance:" ,df["ED"].mean())
    print("Edit distance corrected:" ,df["ED_corrected"].mean())

    # print(dataset.name)
    df.to_json(dataset.parent / f"corrected-{dataset.name}", orient="records", lines=True)
    
    for i in random.sample(range(0, len(df)), 3):
        print("correct\t\t", df.iloc[i]["sentence"])
        print("error\t\t", df.iloc[i]["error"])
        print("masked\t\t", df.iloc[i]["masked"])
        print("replace\t\t", df.iloc[i]["replace"])
        print("corrected\t", df.iloc[i]["corrected"])
        print("ED\t\t", df.iloc[i]["ED_corrected"])
        print("-----------------------")
        print()

Calculating edit distance...


2.555544009144757
3.3618414215940975
correct		 Narodil se v Turčianském Svatém Martině v rodině známého slovenského spisovatele Svetozára HurbanaVajanského
error		 Narodil seu v Turčianském Sv1atém Martně v rodině známého kslovenského spisovatele Svetozára HurbanaVajanského
masked		 Narodil seu v Turčianském [MASK] Martně v rodině známého [MASK] spisovatele Svetozára HurbanaVajanského
replace		 ['Svatém', 'slovenského']
corrected	 Narodil seu v Turčianském Svatém Martně v rodině známého slovenského spisovatele Svetozára HurbanaVajanského
ED		 2
-----------------------

correct		 Dne 10 listopadu 1992 přijalo Národní shromáždění Ázerbájdžánu výnos prezidenta Abulfazem Elčibejem č
error		 Dne 10 listopadu 1992 přijalo Národní shromáždění Ázerbájdžánu výnos prezidenta Abulfazem Elčibejem č
masked		 None
replace		 [None]
corrected	 None
ED		 0
-----------------------

correct		 Teplota během portrétování byla kolem 10 °C
error		 Teplota během portrétování byla kolem 10 °C
masked		 None
rep

2.555544009144757
6.216876234022654
correct		 Vyšší sekundární vzdělávání v zemích EU
error		 Všš sekudární vzdělávání vh zemíczemích E
masked		 [MASK] [MASK] vzdělávání [MASK] [MASK] [MASK]
replace		 ['-', 'š', 'v', 'h', 'ČR']
corrected	 - š vzdělávání v h ČR
ED		 22
-----------------------

correct		 Historie
error		 HstoiJHstoiJe6
masked		 [MASK]
replace		 ['[SEP]']
corrected	 [SEP]
ED		 8
-----------------------

correct		 užíval až v letech 1262–1269 takže například historička Jana Zachová datuje originální privilegium až do roku 1262
error		 užíval až ev ltech 1262–1269 takže například historička Jana Zchová datuje originální privil3egim až o roku 1262
masked		 užíval až [MASK] ltech 1262–1269 takže například historička Jana Zchová datuje originální [MASK] až [MASK] roku 1262
replace		 ['v', 'verzi', 'do']
corrected	 užíval až v ltech 1262–1269 takže například historička Jana Zchová datuje originální verzi až do roku 1262
ED		 11
-----------------------

